# LOAD LIBRARIES

In [1]:
import mlflow
import mlflow.xgboost
import mlflow.data
from mlflow.data.pandas_dataset import PandasDataset

import xgboost as xgb
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd
import os

# SETTINGS

In [2]:
mlflow_arn = "arn:aws:sagemaker:eu-west-1:575618486322:mlflow-tracking-server/dev-mlflow"
mlflow_experiment_name = "02-sample-experiment"

# SET MLFLOW

In [3]:
mlflow.set_tracking_uri(mlflow_arn)
mlflow.set_experiment(mlflow_experiment_name)

<Experiment: artifact_location='s3://ipf-sds-datalake-dev-data-science-bucket/mlflow/2', creation_time=1730285259326, experiment_id='2', last_update_time=1730285259326, lifecycle_stage='active', name='02-sample-experiment', tags={}>

# LOAD DATA

In [4]:
# Load Diabetes dataset
data = load_diabetes()
X = data.data
y = data.target

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

df_train = pd.DataFrame(data=X_train, columns=data.feature_names)
df_train['target'] = y_train

df_test = pd.DataFrame(data=X_test, columns=data.feature_names)
df_test['target'] = y_test

# LOG NESTED MODELS

In [8]:
param = [1, 2, 3, 4, 5]

with mlflow.start_run(run_name = "nested-run") as run:
    for p in param:
        with mlflow.start_run(nested=True, run_name = f"my-param{p}") as child_run:
            mlflow.log_param("max_depth", p)

            model = xgb.XGBRegressor(n_estimators=100, max_depth=p, random_state=42)
            model.fit(X_train, y_train)

            # Make predictions
            y_pred = model.predict(X_test)

            # Calculate metrics
            mse = mean_squared_error(y_test, y_pred)

            mlflow.log_metric("mse", mse)
            
            mlflow.xgboost.log_model(model, "model")

print("Logging completed.")

/opt/conda/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [11:13:22] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1713397725960/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support fo

Logging completed.
